In [ ]:
%matplotlib inline

from beras.data_utils import HDF5Tensor
import matplotlib.pyplot as plt
import numpy as np
from deepdecoder.utils import visualise_tiles, zip_visualise_tiles, np_binary_mask
from deepdecoder.networks import dcgan_generator, dcgan_discriminator
from deepdecoder.data import generate_grids
from deepdecoder.model_utils import add_uniform_noise, plot_weights_histogram
from deepdecoder.grid_curriculum import get_generator_and_callback, reduced_id_lecture, exam, \
    z_rot_lecture, y_rot_lecture, x_rot_lecture
from deepdecoder.visualise import plt_hist
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import Callback
import pylab
import time

pylab.rcParams['figure.figsize'] = (18, 18)
import theano


In [ ]:
g = dcgan_generator(input_dim=50, nb_output_channels=1)

In [ ]:
weights_fname = "generator_pyramdi_loss.hdf5"
g.load_weights(weights_fname)

In [ ]:
start = time.time()
g.compile('adam', 'mse')
print("Compiling done in {}s".format(time.time() - start))

In [ ]:
def weights_histogram(model, bins=50):                                          
    hists = []                                                                  
    for i, layer in enumerate(model.layers):                                    
        name = str(type(layer)) + "_{}".format(i)                               
        weights = layer.get_weights()                                           
        for wi, weight in enumerate(weights):                                   
            plt_hist(weight, name)
            plt.show()
    return hists 
weights_histogram(g)

In [ ]:
dense = g.layers[0]
weight = dense.get_weights()[0]
print(weight.shape)
n = 8
plt.imshow(weight.reshape((50*n, 4096//n)))
plt.colorbar()
plt.show()

In [ ]:
def visualize():
    generator = gen_diff_gan(128)
    input_dim = 40
    batch = next(generator)
    params = batch.params
    z = np.random.uniform(-1, 1, (params.shape[0], input_dim - params.shape[1]))
    grids = g.predict(np.concatenate([params, z], axis=1).astype(np.float32))
    zip_visualise_tiles(
        batch.grid_bw,
        grids[:, 0],
        grids[:, 1]
       )

In [ ]:
visualize()

In [ ]:
for std in (0.01, 0.03, 0.04, 0.05, 0.06, 0.07, 0.1):
    g.load_weights(weights_fname)
    print(std)
    add_uniform_noise(g, std)
    visualize()